In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Parameters
num_features = 10       # Number of input features in the sequence
time_steps = 45         # Time steps in the input sequence
forecast_steps = 10     # Number of forecast steps (output sequence length)
num_locations = 4       # Number of locations
hidden_size = 64        # GRU hidden size
num_samples = 1000      # Number of data samples

# Generate random data
np.random.seed(42)
X_sequence = np.random.rand(num_samples, time_steps, num_locations, num_features).astype(np.float32)
resistance = np.random.rand(num_samples, 1).astype(np.float32)  # Single value per sample
power = np.random.rand(num_samples, 1).astype(np.float32)       # Single value per sample
torque = np.random.rand(num_samples, 1).astype(np.float32)      # Single value per sample
total_energy = np.random.rand(num_samples, 1).astype(np.float32)  # Single value per sample

# Convert data to PyTorch tensors
X_sequence = torch.tensor(X_sequence)
resistance = torch.tensor(resistance)
power = torch.tensor(power)
torque = torch.tensor(torque)
total_energy = torch.tensor(total_energy)

# Define the Seq2Seq Forecast model with GRU encoder and decoder
class ForecastModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ForecastModel, self).__init__()
        self.encoder_gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.decoder_gru = nn.GRU(output_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, forecast_steps):
        batch_size, time_steps, num_locations, num_features = x.size()
        
        # Initialize an empty tensor for storing forecasts across locations
        forecasts = []

        for location in range(num_locations):
            # Process each location independently
            location_input = x[:, :, location, :]  # Shape: (batch_size, time_steps, num_features)
            
            # Encoder
            _, hidden = self.encoder_gru(location_input)
            
            # Decoder
            decoder_input = torch.zeros(batch_size, 1, num_features).to(x.device)
            outputs = []
            
            for _ in range(forecast_steps):
                out, hidden = self.decoder_gru(decoder_input, hidden)
                out = self.fc(out.squeeze(1))  # Shape: (batch_size, output_size)
                outputs.append(out)
                decoder_input = out.unsqueeze(1)
            
            # Concatenate outputs for each forecast step
            location_forecast = torch.stack(outputs, dim=1)  # Shape: (batch_size, forecast_steps, num_features)
            forecasts.append(location_forecast)

        # Stack forecasts for all locations
        forecasts = torch.stack(forecasts, dim=2)  # Shape: (batch_size, forecast_steps, num_locations, num_features)
        return forecasts

# Define the combined DNN model for total energy prediction
class EnergyPredictionModel(nn.Module):
    def __init__(self, forecast_output_size, static_input_size, hidden_size):
        super(EnergyPredictionModel, self).__init__()
        self.forecast_model = ForecastModel(input_size=num_features, hidden_size=hidden_size, output_size=forecast_output_size)
        
        # Feed-forward network for total energy prediction
        self.fc1 = nn.Linear(forecast_output_size * forecast_steps * num_locations + static_input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Predict a single total energy value
    
    def forward(self, x_seq, resistance, power, torque, forecast_steps):
        # Forecast model to get the output sequence
        forecast_out = self.forecast_model(x_seq, forecast_steps)  # Shape: (batch_size, forecast_steps, num_locations, num_features)
        forecast_out_flat = forecast_out.view(forecast_out.size(0), -1)  # Flatten the forecast output across locations and time steps
        
        # Repeat static inputs to match flattened forecast dimensions
        static_inputs = torch.cat((resistance, power, torque), dim=1)  # Shape: (batch_size, 3)
        
        # Concatenate static inputs with the flattened forecast output
        combined_input = torch.cat((forecast_out_flat, static_inputs), dim=1)
        
        # Feed-forward layers for total energy prediction
        x = torch.relu(self.fc1(combined_input))
        x = torch.relu(self.fc2(x))
        energy_pred = self.fc3(x)  # Output: (batch_size, 1)
        
        return energy_pred

# Initialize the model, loss function, and optimizer
model = EnergyPredictionModel(forecast_output_size=num_features, static_input_size=3, hidden_size=hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    energy_pred = model(X_sequence, resistance, power, torque, forecast_steps)
    loss = criterion(energy_pred, total_energy)  # Adjusted to match prediction dimensions
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

